In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

# Coordinates vs tile_number

In [ ]:
def to_coordinates(tile_number, N):
    coordinates = (tile_number%N, tile_number//N)
    return coordinates

def to_tile_number(x, y, N):
    return x + y*N

# Transformations

In [ ]:
def rotation(N, tile_number):
    (x, y) = to_coordinates(tile_number, N)
    return to_tile_number(N-1 - y, x, N)

def double_rotation(N, tile_number):
    return rotation(N, rotation(N, tile_number))

def triple_rotation(N, tile_number):
    return rotation(N, double_rotation(N, tile_number))
    
def mirror(N, tile_number):
    (x, y) = to_coordinates(tile_number, N)
    return to_tile_number(N-1 - x, y, N)

def mirror_rotation(N, tile_number):
    return rotation(N, mirror(N, tile_number))

def mirror_double_rotation(N, tile_number):
    return double_rotation(N, mirror(N, tile_number))

def mirror_triple_rotation(N, tile_number):
    return triple_rotation(N, mirror(N, tile_number))

def all_transformations():
    return [rotation, double_rotation, triple_rotation,
            mirror, mirror_rotation, mirror_double_rotation, mirror_triple_rotation]

## Plot chess board (with pieces)

In [ ]:
def plot_chess(N, pieces = []):
    # Make a NxN grid...
    image = np.zeros(N*N)

    # Set every 'other' cell to one
    whiteCells = [x+y*N for y in range(0, N) for x in range(y%2, N, 2)]
    image[whiteCells] = np.ones(N*N //2 + N%2)

    # show pieces
    image[pieces] = np.ones(len(pieces)) * 0.5

    image = image.reshape((N, N))
    row_labels = range(N, 0, -1)
    col_labels = [chr(65+i) for i in range(N)]
    plt.imshow(image)
    plt.xticks(range(N), col_labels)
    plt.yticks(range(N), row_labels)

## Solutions class

In [ ]:
class Solution:
    
    def __init__(self, pieces):
        self.__pieces = sorted(pieces)
        self.__N = len(pieces)
    
    def is_fundamentally_different(self, others):
        for other in others:
            if self.__is_transformation_of(other):
                return False
        return True
    
    def __is_transformation_of(self, other):
        for transformation in all_transformations():
            if self.__transform(transformation) == other:
                return True
        return False
    
    def __transform(self, transformation):
        new_pieces = [transformation(self.__N, piece) for piece in self.__pieces]
        return Solution(new_pieces)
    
    def plot(self):
        plot_chess(self.__N, self.__pieces)
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__pieces == other.__pieces
        else:
            return False
    
    # __str__ would be more appropriate, but [] redirects str() to repr()
    def __repr__(self):
        return 'Solution(' + str(self.__pieces) + ')'

## Solver class

In [ ]:
class Solver:
    def __init__(self, N):
        self.__N = N
        self.__all_solutions = []

    def solve_queens_problem(self):
        for column in range(self.__N):
            queens = [column]
            self.__try_next_queen(queens)
        return self.__all_solutions

    def __try_next_queen(self, queens):
        queen = len(queens)
        for column in range(self.__N):
            queen_tile_number = queen*self.__N + column

            if self.__spot_free(queens, queen_tile_number):
                queens.extend([queen_tile_number])
                self.__try_next_queen(queens)
                if len(queens) >= self.__N:
                    self.__all_solutions.append(Solution(queens[:]))
                queens.pop()

    def __spot_free(self, queens, spot):
        for queen in queens:
            if self.__queen_covers(spot, queen):
                return False
        return True
        
    def __queen_covers(self, queen, spot):
        (x_queen, y_queen) = to_coordinates(queen, self.__N)
        (x_spot, y_spot) = to_coordinates(spot, self.__N)
        if x_queen == x_spot:
            return True
        if abs(x_queen - x_spot) == abs(y_queen - y_spot):
            return True
        return False

## Post processing

In [ ]:
def remove_symmetries(solutions):
    filtered_solutions = []
    for solution in solutions:
        if solution.is_fundamentally_different(filtered_solutions):
            filtered_solutions.append(solution)
    return filtered_solutions

def show_all_solutions(solutions):
    if len(solutions) == 0:
        print('No solutions where found.')
        return

    print('The solutions are: ' + str(solutions))

    grid_size = math.ceil(math.sqrt(len(solutions)))

    plt.figure(figsize=(3*grid_size, 3*grid_size))

    for i, solution in enumerate(solutions):
        plt.subplot(grid_size, grid_size, i+1)
        solution.plot()

    plt.show()

## Run for all N's in range

In [ ]:
def run():
    for N in range(9):
        print('for N = ' + str(N))
        all_solutions = Solver(N).solve_queens_problem()
        different_solutions = remove_symmetries(all_solutions)
        show_all_solutions(different_solutions)
        
run()